<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Portugués a Ingles: ASR Baseline Whisper Medium**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# path = '/content/drive/MyDrive/Colab Notebooks/TA/A3/'

Mounted at /content/drive


In [2]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import whisper
from whisper.normalizers.basic import BasicTextNormalizer

from tqdm.notebook import tqdm
import pandas as pd

import jiwer

model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 117MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

<p style="page-break-after:always;"></p>

In [4]:
# import tarfile

# tar_path = "/content/drive/MyDrive/Colab Notebooks/TA/A3/pt.tar"
output_dir = "/content/drive/MyDrive/Colab Notebooks/TA/A3/pt"

# with tarfile.open(tar_path, 'r') as tar:
#     tar.extractall(path=output_dir)

# print("Extraction complete!")


In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("facebook/covost2", 'pt_en', data_dir=output_dir)

print(raw_datasets)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.4k [00:00<?, ?B/s]

covost2.py:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

The repository for facebook/covost2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/covost2.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/9158 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3318 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4023 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 9158
    })
    validation: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 3318
    })
    test: Dataset({
        features: ['client_id', 'file', 'audio', 'sentence', 'translation', 'id'],
        num_rows: 4023
    })
})


In [6]:
raw_datasets["train"].features

{'client_id': Value(dtype='string', id=None),
 'file': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None),
 'translation': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [7]:
raw_datasets["train"][:5]["file"]

['/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277034.mp3',
 '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277037.mp3',
 '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277041.mp3',
 '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277042.mp3',
 '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277044.mp3']

In [8]:
raw_datasets["train"][:5]["audio"]

[{'path': '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277034.mp3',
  'array': array([-1.81898940e-12,  5.45696821e-12, -2.18278728e-11, ...,
          1.10979299e-05,  2.18208988e-05,  3.43878878e-06]),
  'sampling_rate': 16000},
 {'path': '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277037.mp3',
  'array': array([-1.45519152e-10, -1.16415322e-10, -4.65661287e-10, ...,
         -1.30036278e-06, -6.07408765e-07, -9.71125974e-06]),
  'sampling_rate': 16000},
 {'path': '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277041.mp3',
  'array': array([-1.09139364e-11,  1.81898940e-12,  1.09139364e-11, ...,
         -5.78370464e-06, -1.24623975e-05,  2.79884334e-05]),
  'sampling_rate': 16000},
 {'path': '/content/drive/MyDrive/Colab Notebooks/TA/A3/pt/clips/common_voice_pt_19277042.mp3',
  'array': array([ 1.09139364e-11,  1.90993887e-11,  4.54747351e-12, ...,
         -2.10382859e-06,  2.25497788e-05,  6.0197860

In [9]:
raw_datasets["train"][:5]["sentence"]

['Altere o dispositivo de saída de áudio.',
 'Até a data-limite, prevista no tratado, vários mísseis foram destruídos.',
 'Eu balancei a cabeça.',
 'Adicione The Cranberries Zombie a playlist de clássicas.',
 'Um cinegrafista filmando um jogo de basquete.']

In [10]:
raw_datasets["train"][:5]["translation"]

['Change the audio output device.',
 'By the deadline shown in the treaty, various missiles had been destroyed.',
 'I shook my head.',
 'Add The Cranberries Zombie to the classics playlist.',
 'A cameraman filming a basketball game.']

<p style="page-break-after:always;"></p>

In [11]:
data=raw_datasets["train"][:2000]

In [12]:
hypotheses = []
for sample in data["file"]:
    hypotheses.append((model.transcribe(sample, language="Portuguese"))['text'])

In [13]:
data["hypothesis"] = hypotheses

In [14]:
data["hypothesis"][:5]

[' altere o dispositivo de saída de áudio.',
 ' Até a data limite prevista no tratado, vários mices foram destruídos.',
 ' Eu balancei a cabeça.',
 ' Adsoni de Cranberries Zombie Aplicia de clássicas',
 ' Um cinegrafista filmando um jogo de basquete.']

In [15]:
normalizer = BasicTextNormalizer()

data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["sentence_clean"] = [normalizer(text) for text in data["sentence"]]
data["translation_clean"] = [normalizer(text) for text in data["translation"]]

<p style="page-break-after:always;"></p>

In [16]:
wer = jiwer.wer(list(data["sentence_clean"]), list(data["hypothesis_clean"]))

print(f"WER: {wer * 100:.2f} %")

WER: 6.35 %


In [17]:
dataframe = pd.DataFrame(dict(transcription=data["hypothesis"], sentence=data["sentence"], translation=data["translation"], transcription_clean=data["hypothesis_clean"],  sentence_clean=data["sentence_clean"], translation_clean=data["translation_clean"] ))
pd.set_option('display.max_colwidth', None)
dataframe.head(5)

,transcription,sentence,translation,transcription_clean,sentence_clean,translation_clean
0,altere o dispositivo de saída de áudio.,Altere o dispositivo de saída de áudio.,Change the audio output device.,altere o dispositivo de saída de áudio,altere o dispositivo de saída de áudio,change the audio output device
1,"Até a data limite prevista no tratado, vários mices foram destruídos.","Até a data-limite, prevista no tratado, vários mísseis foram destruídos.","By the deadline shown in the treaty, various missiles had been destroyed.",até a data limite prevista no tratado vários mices foram destruídos,até a data limite prevista no tratado vários mísseis foram destruídos,by the deadline shown in the treaty various missiles had been destroyed
2,Eu balancei a cabeça.,Eu balancei a cabeça.,I shook my head.,eu balancei a cabeça,eu balancei a cabeça,i shook my head
3,Adsoni de Cranberries Zombie Aplicia de clássicas,Adicione The Cranberries Zombie a playlist de clássicas.,Add The Cranberries Zombie to the classics playlist.,adsoni de cranberries zombie aplicia de clássicas,adicione the cranberries zombie a playlist de clássicas,add the cranberries zombie to the classics playlist
4,Um cinegrafista filmando um jogo de basquete.,Um cinegrafista filmando um jogo de basquete.,A cameraman filming a basketball game.,um cinegrafista filmando um jogo de basquete,um cinegrafista filmando um jogo de basquete,a cameraman filming a basketball game


All the data is stored into a file using 'csv' format

In [18]:
dataframe.to_csv('/content/drive/MyDrive/Colab Notebooks/TA/A3/pt_en_ASR_Whisper_Medium_train.csv', encoding='utf-8')